In [31]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from pulp import * 

In [32]:
path = 'data/nutrition.csv'
data = pd.read_csv(path).drop('Unnamed: 0', axis=1)
data.head()

,name,serving_size,calories,total_fat,saturated_fat,cholesterol,sodium,choline,folate,folic_acid,...,fat,saturated_fatty_acids,monounsaturated_fatty_acids,polyunsaturated_fatty_acids,fatty_acids_total_trans,alcohol,ash,caffeine,theobromine,water
0,Cornstarch,100 g,381,0.1g,NaN,0,9.00 mg,0.4 mg,0.00 mcg,0.00 mcg,...,0.05 g,0.009 g,0.016 g,0.025 g,0.00 mg,0.0 g,0.09 g,0.00 mg,0.00 mg,8.32 g
1,"Nuts, pecans",100 g,691,72g,6.2g,0,0.00 mg,40.5 mg,22.00 mcg,0.00 mcg,...,71.97 g,6.180 g,40.801 g,21.614 g,0.00 mg,0.0 g,1.49 g,0.00 mg,0.00 mg,3.52 g
2,"Eggplant, raw",100 g,25,0.2g,NaN,0,2.00 mg,6.9 mg,22.00 mcg,0.00 mcg,...,0.18 g,0.034 g,0.016 g,0.076 g,0.00 mg,0.0 g,0.66 g,0.00 mg,0.00 mg,92.30 g
3,"Teff, uncooked",100 g,367,2.4g,0.4g,0,12.00 mg,13.1 mg,0,0,...,2.38 g,0.449 g,0.589 g,1.071 g,0,0,2.37 g,0,0,8.82 g
4,"Sherbet, orange",100 g,144,2g,1.2g,1mg,46.00 mg,7.7 mg,4.00 mcg,0.00 mcg,...,2.00 g,1.160 g,0.530 g,0.080 g,1.00 mg,0.0 g,0.40 g,0.00 mg,0.00 mg,66.10 g


In [33]:
data = data[['name','calories','carbohydrate','total_fat','protein']]
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8789 entries, 0 to 8788
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          8789 non-null   object
 1   calories      8789 non-null   int64 
 2   carbohydrate  8789 non-null   object
 3   total_fat     8789 non-null   object
 4   protein       8789 non-null   object
dtypes: int64(1), object(4)
memory usage: 343.4+ KB
None


,name,calories,carbohydrate,total_fat,protein
0,Cornstarch,381,91.27 g,0.1g,0.26 g
1,"Nuts, pecans",691,13.86 g,72g,9.17 g
2,"Eggplant, raw",25,5.88 g,0.2g,0.98 g
3,"Teff, uncooked",367,73.13 g,2.4g,13.30 g
4,"Sherbet, orange",144,30.40 g,2g,1.10 g


In [34]:
data['carbohydrate'] = np.array([data['carbohydrate'].tolist()[i].split(' ') for i in range(len(data))])[:,0].astype('float')
data['protein'] = np.array([data['protein'].tolist()[i].split(' ') for i in range(len(data))])[:,0].astype('float')
data['total_fat'] = np.array([data['total_fat'].tolist()[i].split('g') for i in range(len(data))])[:,0].astype('float')

In [35]:
data.head()

,name,calories,carbohydrate,total_fat,protein
0,Cornstarch,381,91.27,0.1,0.26
1,"Nuts, pecans",691,13.86,72.0,9.17
2,"Eggplant, raw",25,5.88,0.2,0.98
3,"Teff, uncooked",367,73.13,2.4,13.30
4,"Sherbet, orange",144,30.40,2.0,1.10


In [36]:
week_days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
split_values = np.linspace(0,len(data),8).astype(int)
split_values[-1] = split_values[-1]-1
def random_dataset():
    frac_data = data.sample(frac=1).reset_index().drop('index',axis=1)
    day_data = []
    for s in range(len(split_values)-1):
        day_data.append(frac_data.loc[split_values[s]:split_values[s+1]])
    return dict(zip(week_days,day_data))

In [37]:
random_dataset()['Monday']

,name,calories,carbohydrate,total_fat,protein
0,"Margarine-like vegetable-oil spread, with adde...",535,0.00,59.0,0.60
1,"Lamb, raw, sweetbread, imported, New Zealand",72,0.00,3.2,11.00
2,"Cheese, nonfat or fat free, Swiss",127,3.40,0.0,28.40
3,"Seaweed, raw, laver",35,5.11,0.3,5.81
4,"Apple juice, with added ascorbic acid, diluted...",47,11.54,0.1,0.14
...,...,...,...,...,...
1251,"Infant formula, liquid concentrate, ADVANCE wi...",128,13.21,7.0,3.13
1252,"Beans, without salt, boiled, cooked, mature se...",127,22.80,0.5,8.67
1253,"Lamb, roasted, cooked, trimmed to 1/8"" fat, se...",190,0.00,8.4,28.60
1254,"Babyfood, strained, creamed, corn, vegetables",57,14.10,0.4,1.40


In [38]:
def build_nutritional_values(kg,calories):
    protein_calories = kg*4
    res_calories = calories-protein_calories
    carb_calories = calories/2.
    fat_calories = calories-carb_calories-protein_calories
    res = {'Protein Calories':protein_calories,'Carbohydrates Calories':carb_calories,'Fat Calories':fat_calories}
    return res

In [39]:
def extract_gram(table):
    protein_grams = table['Protein Calories']/4.
    carbs_grams = table['Carbohydrates Calories']/4.
    fat_grams = table['Fat Calories']/9.
    res = {'Protein Grams':protein_grams, 'Carbohydrates Grams':carbs_grams,'Fat Grams':fat_grams}
    return res

In [40]:
build_nutritional_values(70,2000)

{'Protein Calories': 280,
 'Carbohydrates Calories': 1000.0,
 'Fat Calories': 720.0}

In [41]:
extract_gram(build_nutritional_values(70,2000))

{'Protein Grams': 70.0, 'Carbohydrates Grams': 250.0, 'Fat Grams': 80.0}

In [42]:
days_data = random_dataset()
def model(day,kg,calories):
    G = extract_gram(build_nutritional_values(kg,calories))
    E = G['Carbohydrates Grams']
    F = G['Fat Grams']
    P = G['Protein Grams']
    day_data = days_data[day]
    day_data = day_data[day_data.calories!=0]
    food = day_data.name.tolist()
    c  = day_data.calories.tolist()
    x  = pulp.LpVariable.dicts( "x", indices = food, lowBound=0, upBound=1.5, cat='Continuous', indexStart=[] )
    e = day_data.carbohydrate.tolist()
    f = day_data.total_fat.tolist()
    p = day_data.protein.tolist()
    prob  = pulp.LpProblem( "Diet", LpMinimize )
    prob += pulp.lpSum( [x[food[i]]*c[i] for i in range(len(food))]  )
    prob += pulp.lpSum( [x[food[i]]*e[i] for i in range(len(x)) ] )>=E
    prob += pulp.lpSum( [x[food[i]]*f[i] for i in range(len(x)) ] )>=F
    prob += pulp.lpSum( [x[food[i]]*p[i] for i in range(len(x)) ] )>=P
    prob.solve()
    variables = []
    values = []
    for v in prob.variables():
        variable = v.name
        value = v.varValue
        variables.append(variable)
        values.append(value)
    values = np.array(values).round(2).astype(float)
    sol = pd.DataFrame(np.array([food,values]).T, columns = ['Food','Quantity'])
    sol['Quantity'] = sol.Quantity.astype(float)
    return sol

In [43]:
sol_monday = model('Monday',70,1500)

In [44]:
sol_monday = sol_monday[sol_monday['Quantity']!=0.0]
sol_monday.Quantity = sol_monday.Quantity*100
sol_monday = sol_monday.rename(columns={'Quantity':'Quantity (g)'})
sol_monday

,Food,Quantity (g)
36,"KELLOGG'S, Garlic and Herb Crackers, ALL-BRAN",150.0
71,"Ostrich, cooked, inside leg",77.0
256,"Pork, roasted, cooked, boneless, separable lea...",150.0
615,"Beef, broiled, cooked, all grades, trimmed to ...",16.0
815,"Beef, broiled, cooked, USDA choice, trimmed to...",150.0
1231,"Nuts, with salt added, dry roasted, cashew nuts",150.0
1236,"Fast Foods, meat and skin and breading, Wing, ...",84.0


In [45]:
def model(prob,day,kg,calories):
    G = extract_gram(build_nutritional_values(kg,calories))
    E = G['Carbohydrates Grams']
    F = G['Fat Grams']
    P = G['Protein Grams']
    day_data = days_data[day]
    day_data = day_data[day_data.calories!=0]
    food = day_data.name.tolist()
    c  = day_data.calories.tolist()
    x  = pulp.LpVariable.dicts( "x", indices = food, lowBound=0, upBound=1.5, cat='Continuous', indexStart=[] )
    e = day_data.carbohydrate.tolist()
    f = day_data.total_fat.tolist()
    p = day_data.protein.tolist()
#    prob  = pulp.LpProblem( "Diet", LpMinimize )
    prob += pulp.lpSum( [x[food[i]]*c[i] for i in range(len(food))]  )
    prob += pulp.lpSum( [x[food[i]]*e[i] for i in range(len(x)) ] )>=E
    prob += pulp.lpSum( [x[food[i]]*f[i] for i in range(len(x)) ] )>=F
    prob += pulp.lpSum( [x[food[i]]*p[i] for i in range(len(x)) ] )>=P
    prob.solve()
    variables = []
    values = []
    for v in prob.variables():
        variable = v.name
        value = v.varValue
        variables.append(variable)
        values.append(value)
    values = np.array(values).round(2).astype(float)
    sol = pd.DataFrame(np.array([food,values]).T, columns = ['Food','Quantity'])
    sol['Quantity'] = sol.Quantity.astype(float)
    sol = sol[sol['Quantity']!=0.0]
    sol.Quantity = sol.Quantity*100
    sol = sol.rename(columns={'Quantity':'Quantity (g)'})
    return sol
def total_model(kg,calories):
    result = []
    for day in week_days:
        prob  = pulp.LpProblem( "Diet", LpMinimize )
        print('Building a model for day %s \n'%(day))
        result.append(model(prob,day,kg,calories))
    return dict(zip(week_days,result))

In [46]:
diet = total_model(70,3000)

Building a model for day Monday 

Building a model for day Tuesday 

Building a model for day Wednesday 

Building a model for day Thursday 

Building a model for day Friday 

Building a model for day Saturday 

Building a model for day Sunday 



In [47]:
diet['Monday']

,Food,Quantity (g)
71,"Ostrich, cooked, inside leg",150.0
256,"Pork, roasted, cooked, boneless, separable lea...",150.0
264,"Fast foods, tomato and sweet onion sauce, swee...",121.0
767,"Beans, with salt, drained, boiled, cooked, yel...",114.0
815,"Beef, broiled, cooked, USDA choice, trimmed to...",150.0
1123,"Oil, principal uses popcorn and flavoring vege...",13.0
1236,"Fast Foods, meat and skin and breading, Wing, ...",150.0
